In [1]:
# Basic imports
import gym
from gym.wrappers import FlattenObservation
# import gymnasium as gym
# from gymnasium.wrappers import FlattenObservation

import numpy as np
import os

# Stable baselines 3
import stable_baselines3
from stable_baselines3 import PPO, TD3, SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.td3 import MlpPolicy

print(stable_baselines3.__version__)
print(gym.__version__)

# URDF Envs
from urdfenvs.urdf_common.urdf_env import UrdfEnv
from urdfenvs.robots.generic_urdf import GenericUrdfReacher
from urdfenvs.sensors.full_sensor import FullSensor
from urdfenvs.scene_examples.goal import goal1
from urdfenvs.urdf_common.reward import Reward

1.8.0
0.21.0


pybullet build time: May 20 2022 19:45:31


In [2]:
MODEL_NAME = 'TD3-001'


models_dir = 'models/' + MODEL_NAME
logdir = 'logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [3]:
class InverseDistanceDenseReward(Reward):
    def calculateReward(self, observation: dict) -> float:
        goal = observation['robot_0']['FullSensor']['goals'][1]['position']
        position = observation['robot_0']['joint_state']['position']
        reward = 1.0/np.linalg.norm(goal-position)
        # print(f'🏆 Reward is: {reward}')
        return reward


robots = [
    GenericUrdfReacher(urdf="pointRobot.urdf", mode="vel"),
]

env = UrdfEnv(
    dt=0.01,
    robots=robots,
    render=False,
)

In [4]:
env.add_goal(goal1)
sensor = FullSensor(['position'], ['position', 'size'], variance=0.0)
env.add_sensor(sensor, [0])
env.set_spaces()
env.set_reward_calculator(InverseDistanceDenseReward())
defaultAction = np.array([0.5, -0.0, 0.0])
pos0 = np.array([0.0, 0.1, 0.0])
vel0 = np.array([1.0, 0.0, 0.0])
ob = env.reset(pos=pos0, vel=vel0)
env.shuffle_goals()

# print(f"{gymnasium.envs.registry.keys()}")
# print(gymnasium.__version__)


b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
lidar_sensor_link

/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(


{'goal1': {'weight': 1.0,
  'type': 'staticSubGoal',
  'indices': [0, 1, 2],
  'epsilon': 0.2,
  'is_primary_goal': True,
  'parent_link': 0,
  'child_link': 3,
  'desired_position': [2.1815851256318215, -1.5239955273684822, 0.1],
  'angle': None,
  'low': [-3.0, -3.0, 0.1],
  'high': [3.0, 3.0, 0.1]}}

In [5]:
# env = gym.make("GymV21Environment-v0", env=env)


In [6]:
env = FlattenObservation(env)



In [7]:

print(f"🚀 Action Space Data Type: {env.action_space.sample().dtype}, Observation Space Data Type: {env.observation_space.sample().dtype}")
print(type(env.action_space))

🚀 Action Space Data Type: float64, Observation Space Data Type: float64
<class 'gym.spaces.box.Box'>


In [9]:
class Float32ActionWrapper(gym.Wrapper):

    def __init__(self, env):
        super().__init__(env)
        self.action_space = gym.spaces.Box(
            low=self.env.action_space.low.astype(np.float32),
            high=self.env.action_space.high.astype(np.float32),
            dtype=np.float32,
        )

    def step(self, action):
        action = action.astype(np.float32)
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

In [10]:
wrapped_env = Float32ActionWrapper(env)

In [11]:

print(f"🚀 Action Space Data Type: {wrapped_env.action_space.sample().dtype}, Observation Space Data Type: {env.observation_space.sample().dtype}")
print(type(wrapped_env.action_space))

🚀 Action Space Data Type: float32, Observation Space Data Type: float64
<class 'gym.spaces.box.Box'>


In [12]:
wrapped_env.action_space

Box([-2.175 -2.175 -2.175], [2.175 2.175 2.175], (3,), float32)

In [13]:
# env = gym.make("GymV21Environment-v0", env=env)


In [14]:
# n_actions = env.action_space.shape[-1]
# action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

TIMESTEPS = 100000
model = TD3(
    MlpPolicy, 
    wrapped_env,
    # action_noise=action_noise,
    verbose=1, 
    # tensorboard_log=logdir,
    # policy_kwargs={"net_arch": [64, 64]},
        )
model.learn(total_timesteps=TIMESTEPS, 
            log_interval=10,
            tb_log_name=MODEL_NAME, 
            progress_bar=False, 
        )
# model.save(f"{MODEL_NAME}-model")
print("🏁 Training complete!")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
lidar_sensor_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
lidar_sensor_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
lidar_sensor_linkb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, lo

KeyboardInterrupt: 